In [1]:
import os, random, numpy as np
import tensorflow as tf
import keras_tuner as kt
import matplotlib
import matplotlib.pyplot as plt

import utils  # your local utils.py

from sklearn.utils import class_weight
from tensorflow.keras import mixed_precision

import shutil
import os

from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras import layers

num_classes = 9

# Set up reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)


# Use mixed precision and GPU memory management like tiny.py
mixed_precision.set_global_policy('mixed_float16')
matplotlib.use("Agg")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_memory_growth(gpus[0], True)

logs_dir = "./logs"

# Check if logs directory exists
if os.path.exists(logs_dir):
    print(f"Clearing logs directory: {logs_dir}")
    shutil.rmtree(logs_dir)  # delete the folder and all its contents
else:
    print(f"No existing logs directory found at: {logs_dir}")




2025-07-15 17:18:55.335821: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752592735.355832   49228 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752592735.361561   49228 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752592735.376532   49228 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752592735.376553   49228 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752592735.376556   49228 computation_placer.cc:177] computation placer alr

No existing logs directory found at: ./logs


In [2]:
IMG_SIZE = 128
BATCH = 2

TRAIN_CSV = "./split_data/train.csv"
VAL_CSV = "./split_data/val.csv"
TEST_CSV = "./split_data/test.csv"

train_paths, train_labels, train_label_encoder = utils.load_paths_and_labels(TRAIN_CSV)
val_paths, val_labels, val_label_encoder = utils.load_paths_and_labels(VAL_CSV)
test_paths, test_labels, test_label_encoder = utils.load_paths_and_labels(TEST_CSV)

# Optional: SAMPLE tiny dataset for FAST testing
# train_paths, train_labels = train_paths[:20], train_labels[:20]
# val_paths, val_labels = val_paths[:10], val_labels[:10]

# Show class distributions
utils.show_dataset_class_distribution("TRAIN", train_labels)
utils.show_dataset_class_distribution("VAL  ", val_labels)
utils.show_dataset_class_distribution("TEST ", test_labels)

# Build datasets
augment_layer = utils.make_augment()
train_ds = utils.make_dataset(train_paths, train_labels, BATCH, shuffle=True, autotune=True)
train_ds = train_ds.map(lambda imgs, labs: (augment_layer(imgs, training=True), labs), num_parallel_calls=tf.data.AUTOTUNE)
val_ds = utils.make_dataset(val_paths, val_labels, BATCH, shuffle=False, autotune=True)


TRAIN {np.int64(0): 276, np.int64(1): 246, np.int64(2): 252, np.int64(3): 474, np.int64(4): 297, np.int64(5): 300, np.int64(6): 552, np.int64(7): 190, np.int64(8): 261}  (total: 2848)
VAL   {np.int64(0): 92, np.int64(1): 82, np.int64(2): 84, np.int64(3): 158, np.int64(4): 99, np.int64(5): 100, np.int64(6): 184, np.int64(7): 64, np.int64(8): 87}  (total: 950)
TEST  {np.int64(0): 93, np.int64(1): 83, np.int64(2): 84, np.int64(3): 158, np.int64(4): 99, np.int64(5): 100, np.int64(6): 185, np.int64(7): 64, np.int64(8): 88}  (total: 954)


I0000 00:00:1752592738.509488   49228 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 84 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


In [3]:
cw_vals = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_w = dict(enumerate(cw_vals))


In [4]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3)))

    # Convolutional layers
    for i in range(hp.Int("conv_blocks", 3, 5, default=4)):
        filters = hp.Choice(f"filters_{i}", values=[32, 64, 128, 256], default=64)
        model.add(layers.Conv2D(filters, (3,3), padding="same"))
        model.add(layers.BatchNormalization())
        model.add(layers.LeakyReLU(alpha=0.1))
        model.add(layers.MaxPooling2D(2,2))

    model.add(layers.GlobalAveragePooling2D())

    # Dense layers
    dense_units = hp.Int("dense_units", min_value=64, max_value=256, step=32, default=128)
    model.add(layers.Dense(dense_units))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    dropout_rate = hp.Float("dropout", min_value=0.2, max_value=0.5, step=0.05, default=0.3)
    model.add(layers.Dropout(dropout_rate))

    # Final layer
    model.add(layers.Dense(num_classes, activation='softmax', dtype='float32'))

    # Compile
    lr = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log", default=1e-3)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model



In [5]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=5,   # FAST: reduce for tests
    factor=3,
    directory='logs',
    project_name='RealWaste_quick_test'
)


/home/jarus/miniconda3/envs/MLP/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [6]:
# Suppose your validation loss is around 0.5
initial_val_loss = 0.5
min_delta_percent = 0.1
min_delta_absolute = min_delta_percent * initial_val_loss  # = 0.05

early_stop_cb = EarlyStopping(
    monitor='val_loss',    
    patience=5,            # stop after 5 epochs with no sufficient improvement
    min_delta=min_delta_absolute,
    mode='min',            
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)


In [7]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir='logs/tensorboard', histogram_freq=1)

tuner.search(
    train_ds,
    validation_data=val_ds,
    class_weight=class_w,
    epochs=50,
    callbacks=[early_stop_cb,
                reduce_lr,
                tensorboard_cb,
                utils.checkpoint_cb(),
                utils.csv_logger_cb()
            ]
)


Trial 2 Complete [00h 00m 06s]

Best val_accuracy So Far: None
Total elapsed time: 00h 00m 11s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
3                 |4                 |conv_blocks
64                |64                |filters_0
32                |256               |filters_1
32                |64                |filters_2
256               |32                |filters_3
160               |224               |dense_units
0.25              |0.4               |dropout
0.0098403         |0.0010144         |learning_rate
128               |None              |filters_4
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
1                 |1                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


2025-07-15 17:19:16.315525: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 16.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-07-15 17:19:16.327798: W tensorflow/core/framework/op_kernel.cc:1857] OP_REQUIRES failed at xla_ops.cc:591 : UNKNOWN: Failed to determine best cudnn convolution algorithm for:
%cudnn-conv-bias-activation.9 = (f32[2,64,128,128]{3,2,1,0}, u8[0]{0}) custom-call(f32[2,3,128,128]{3,2,1,0} %bitcast.7018, f32[64,3,3,3]{3,2,1,0} %bitcast.6568, f32[64]{0} %bitcast.7653), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/conv2d_1/convolution" source_file="/home/jarus/miniconda3/envs/MLP/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200},

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/home/jarus/miniconda3/envs/MLP/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/jarus/miniconda3/envs/MLP/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jarus/miniconda3/envs/MLP/lib/python3.12/site-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jarus/miniconda3/envs/MLP/lib/python3.12/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(

In [ ]:
best_hp = tuner.get_best_hyperparameters(1)[0]
print("Best hyperparameters found:")
print(f"Conv Blocks: {best_hp.get('conv_blocks')}")
for i in range(best_hp.get('conv_blocks')):
    print(f"Filters_{i}: {best_hp.get(f'filters_{i}')}")
print(f"Dense units: {best_hp.get('dense_units')}")
print(f"Dropout: {best_hp.get('dropout')}")
print(f"Learning rate: {best_hp.get('learning_rate')}")


In [ ]:
best_model = tuner.hypermodel.build(best_hp)
history = best_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[early_stop_cb, reduce_lr]
)
